In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
seq = pd.read_csv('../input/protein-data-set/pdb_data_seq.csv')
#This loads a set of pdb data from kaggle

seq.head(10)

In [ ]:
seq.describe()

In [ ]:
seq = seq.dropna(axis=0)
#This gets rid of entries which are missing data
seq.describe()


In [ ]:
structure = pd.read_csv('../input/protein-data-set/pdb_data_no_dups.csv')
structure.head()
#This loads a second set of pdb data from kaggle

In [ ]:
structure.describe()

In [ ]:
structure=structure.dropna(axis=0)
#This gets rid of entries in that second data set which are missing data
structure.describe()


In [ ]:
df = seq.set_index('structureId').merge(structure.set_index('structureId'),on='structureId',how='left')
df = df.reset_index()
df.head()
df.describe()

In [ ]:
y=structure.resolution
prediction_features = ['residueCount', 'structureMolecularWeight', 'crystallizationTempK', 'phValue','publicationYear', 'densityMatthews', 'densityPercentSol', ]
X=structure[prediction_features]
X.describe
#This selects the features of the database to look at...we're predicting resolution based on a bunch of different factors

In [ ]:
X.head()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


    

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)
#This gives me a train set and a test set of the data

In [ ]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
#This makes a decision tree using the above chosen categories and returns the mean absolute error

In [ ]:
get_mae(2, train_X, val_X, train_y, val_y)
#This is the MAE for 2 nodes for comparison

In [ ]:
for max_leaf_nodes in [5, 50, 100, 500, 1000, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, my_mae))
#seemingly 500 nodes is the best

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
struct_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, struct_preds))
#This runs a random forest model and returns the MAE for that model, which is even better than the 500 nodes. 

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(forest_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())
#This checks permutation importance; clearly the bottom three don't matter much. I'm not surprised molecular weight (or residue count) affects resolution... what is surprising is that molecular weight is so much more important than residue count or the various densities.

In [ ]:
from sklearn import preprocessing
from sklearn import utils
import graphviz
lab_enc = preprocessing.LabelEncoder()

encoded = lab_enc.fit_transform(val_y)
tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(val_X, encoded)
tree_graph = tree.export_graphviz(tree_model, out_file=None, feature_names=prediction_features)
graphviz.Source(tree_graph)

In [ ]:
#from matplotlib import pyplot as plt
#from pdpbox import pdp, get_dataset, info_plots

## Create the data that we will plot
#pdp_structuremolecularweight = pdp.pdp_isolate(model=tree_model, dataset=val_X, model_features=prediction_features, feature='structureMolecularWeight')

## plot it
#pdp.pdp_plot(pdp_structuremolecularweight, 'structureMolecularWeight')
#plt.show()
#DOESN'T WORK BECAUSE THE PLOT IS TOO LARGE AND I DON'T KNOW HOW TO FIX THAT